# Calculating the number of college vacancies in USP

This notebook is meant to be run on Google Colab so that the user does not have to install and maintain a Java environment just for it. Therefore, if you do not have Java already installed in your system, it is **highly recommended that you run in Colab**

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/tomaz-suller/manual-do-vestibulando/blob/main/src/notebooks/0-Calculate%20vacancies%20in%20USP.ipynb)

## A note about `tabula-py`

`tabula-py` is a tool for converting PDF tables to pandas DataFrame. `tabula-py` is a wrapper of [`tabula-java`](https://github.com/tabulapdf/tabula-java), which requires Java on your machine. `tabula-py` also enables you to convert tables in a PDF into CSV/TSV files.

`tabula-py`'s PDF extraction accuracy is same as `tabula-java` or [tabula app](https://tabula.technology/); GUI tool of tabula.

`tabula-py` is good for:
- automation with Python scripts;
- advanced analytics after conversion;
- casual analytics with Jupyter notebook or Google Colab.


## Environment setup

### Tabula setup

`tabula-py` requires a java environment, so let's check the java environment on your machine.

In [1]:
!java -version

openjdk version "11.0.11" 2021-04-20
OpenJDK Runtime Environment (build 11.0.11+9-Ubuntu-0ubuntu2.18.04)
OpenJDK 64-Bit Server VM (build 11.0.11+9-Ubuntu-0ubuntu2.18.04, mixed mode, sharing)


After confirming the java environment, install tabula-py by using pip.

In [2]:
!pip install -q tabula-py

     |████████████████████████████████| 12.0 MB 5.6 MB/s 


Before trying tabula-py, check your environment via tabula-py `environment_info()` function, which shows Python version, Java version, and your OS environment.

In [3]:
import tabula

tabula.environment_info()

Python version:
    3.7.12 (default, Sep 10 2021, 00:21:48) 
[GCC 7.5.0]
Java version:
    openjdk version "11.0.11" 2021-04-20
OpenJDK Runtime Environment (build 11.0.11+9-Ubuntu-0ubuntu2.18.04)
OpenJDK 64-Bit Server VM (build 11.0.11+9-Ubuntu-0ubuntu2.18.04, mixed mode, sharing)
tabula-py version: 2.3.0
platform: Linux-5.4.144+-x86_64-with-Ubuntu-18.04-bionic
uname:
    uname_result(system='Linux', node='cc1203dc36b0', release='5.4.144+', version='#1 SMP Tue Dec 7 09:58:10 PST 2021', machine='x86_64', processor='x86_64')
linux_distribution: ('Ubuntu', '18.04', 'bionic')
mac_ver: ('', ('', '', ''), '')
    


### Python environment setup

In [4]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import tabula

## Convert the PRG PDF into a usable Pandas `DataFrame`

The PDF comes from USP's dean's office website. We will read its table into a Pandas `DataFrame` for further processing.

The PDF has been manually augmented by adding vertical lines to aid OCR.

`tabula-py`'s `read_pdf()` returns a list of `DataFrame`s by default.

In [249]:
PDF_PATH = '../../data/1_raw/vagas/2022.pdf'
NUMBER_OF_PAGES = 7

page_dfs = []
for i in range(NUMBER_OF_PAGES):
  page_df = tabula.read_pdf(PDF_PATH, 
                            lattice=True, 
                            pandas_options={'header': None}, 
                            pages=str(i+1),
                            multiple_tables=True)
  # tabula detects two identical tables per page, so we only
  # take the first one
  page_dfs.append(page_df[0])

print(len(page_dfs))

7


In [203]:
raw_df = page_dfs[0].copy(deep=True)
example_df = raw_df.copy(deep=True)
example_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24
0,FuvestSISU\rUnidadeÁreaCursoEPPPI\rACEPPPITota...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Unidade,Área,Curso,NaN,NaN,NaN,Fu,NaN,st,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S,SU,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,AC,NaN,EP,NaN,PPI,Total,NaN,AC,NaN,NaN,EP,NaN,NaN,NaN,NaN,PPI,NaN,NaN,NaN,Total,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,L1,NaN,L3,NaN,SubTot,L2,NaN,L4,NaN,SubTot,NaN,NaN,NaN
4,FFCLRP\rFFCLRP\rFFCLRP\rFFCLRP\rFFCLRP\rFFCLRP...,E,Química - Bacharelado – Habilitações:\rQuímica...,NaN,21,NaN,13,NaN,8,42,NaN,9,0,NaN,5,NaN,5,0,NaN,4,NaN,4,NaN,18,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,FM,NaN,Subtotal Unidade,NaN,85,NaN,57,NaN,34,176,NaN,38,0,NaN,22,NaN,22,0,NaN,14,NaN,14,NaN,74,NaN
71,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
72,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
73,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Several steps are required for transforming the resulting PDF into a useful format:

### Remove header

Firstly, each pages's first rows comes from its header, which is useless for our analysis, and the first column may be filled with NAs, so we remove them with `slice_from_df`.


In [192]:
def slice_from_df(df, init_row=4, init_col=1):
  clean_df = df
  if init_row is not None and init_col is not None:
    clean_df = df.iloc[init_row:, init_col:] 
  elif init_row is not None:
    clean_df = df.iloc[init_row:, :]
  elif init_col is not None:
    clean_df = df.iloc[:, init_col:]
  return clean_df.reset_index(drop=True)

In [193]:
slice_from_df(example_df, init_col=None)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24
0,FFCLRP\rFFCLRP\rFFCLRP\rFFCLRP\rFFCLRP\rFFCLRP...,E,Química - Bacharelado – Habilitações:\rQuímica...,NaN,21,NaN,13,NaN,8,42,NaN,9,0,NaN,5,NaN,5,0,NaN,4,NaN,4,NaN,18,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,FM,NaN,Subtotal Unidade,NaN,85,NaN,57,NaN,34,176,NaN,38,0,NaN,22,NaN,22,0,NaN,14,NaN,14,NaN,74,NaN
67,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
68,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
69,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Remove columns without data

Secondly, some spurious columns, which contain many NAs, remain, so we remove them with `remove_cols_without_data`.

In [267]:
def remove_cols_without_data(df, threshold=0.9, init_data_col=3):
  unmodified_df_cols = df.iloc[:,:init_data_col]
  clean_data_cols = df.iloc[:, init_data_col:] \
    .dropna(axis=1, thresh=int(df.shape[0]*(1-threshold)))
  return pd.concat([unmodified_df_cols,
                    clean_data_cols],
                   axis=1, ignore_index=True)

In [258]:
remove_cols_without_data(example_df).head()

                                                    0  ...                                                  2
0                                                 NaN  ...                                                NaN
1   FFCLRP\rFFCLRP\rFFCLRP\rFFCLRP\rFFCLRP\rFFCLRP...  ...  Química - Bacharelado – Habilitações:\rQuímica...
2                                                 NaN  ...                                                NaN
3                                                 NaN  ...                                                NaN
4                                                 NaN  ...                                                NaN
..                                                ...  ...                                                ...
67                                                 FM  ...                                   Subtotal Unidade
68                                                NaN  ...                                                NaN
69        

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,L1,L3,SubTot,L2,L4,SubTot,NaN
1,FFCLRP\rFFCLRP\rFFCLRP\rFFCLRP\rFFCLRP\rFFCLRP...,E,Química - Bacharelado – Habilitações:\rQuímica...,21,13,8,42,9,0,5,5,0,4,4,18
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [246]:
example_df = remove_cols_without_data(slice_from_df(raw_df, init_row=3, init_col=None))
example_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,L1,L3,SubTot,L2,L4,SubTot,NaN
1,FFCLRP\rFFCLRP\rFFCLRP\rFFCLRP\rFFCLRP\rFFCLRP...,E,Química - Bacharelado – Habilitações:\rQuímica...,21,13,8,42,9,0,5,5,0,4,4,18
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,FM,NaN,Subtotal Unidade,85,57,34,176,38,0,22,22,0,14,14,74
68,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
69,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
70,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Fix institutes

Thirdly, we correctly assign each course to its respective institute with `split_institutes` and `fix_institute_column`, exploiting the fact that each insitute is followed by a row filled with NAs.

In [223]:
def split_institutes(df):
  previous_cut = 0
  dfs = []
  for key, row in df.iterrows():
    if row.isna().all() and key > previous_cut:
      dfs.append(df.iloc[previous_cut:key,:])
      previous_cut = key + 1
    
  return dfs

In [224]:
for institute_df in split_institutes(example_df):
  print(institute_df)

                                                  0  1   ... 13  14
0  FFCLRP\rFFCLRP\rFFCLRP\rFFCLRP\rFFCLRP\rFFCLRP...  E  ...  4  18

[1 rows x 15 columns]
    0    1    2    3    4    5    6    7    8    9    10   11   12   13   14
2  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN
    0    1    2    3    4    5    6    7    8    9    10   11   12   13   14
4  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN
    0    1                             2    3    4   ...   10   11   12   13   14
6  NaN  NaN                           NaN  NaN  NaN  ...  NaN  NaN  NaN  NaN  NaN
7  NaN    E  Bacharelado em Física Médica   13    9  ...    3    0    2    2   12

[2 rows x 15 columns]
    0  1                                      2   3  4  5   ... 9  10 11 12 13  14
9  NaN  E  Licenciatura e Bacharelado em Química  14  8  6  ...  4  4  0  2  2  12

[1 rows x 15 columns]
     0  1                                               2   3   ... 11 12 1

In [225]:
def fix_institute_column(df, institute_column=0):
  dfs = split_institutes(df)
  fixed_df = []
  for i in range(len(dfs)):
    institute_df = dfs[i]
    for j in range(institute_df.shape[0]):
      if not pd.isna(institute_df.iloc[j, institute_column]):
        institute_name = str(
            institute_df.iloc[j, institute_column]).split('\r')[0]
        break
    if i == len(dfs)-1:
      # Throw away broken institute column
      institute_df = institute_df.shift(axis=1)
    institute_df.iloc[:,institute_column] = institute_name
    fixed_df.append(institute_df)
  return pd.concat(fixed_df, axis=0, ignore_index=True)



In [227]:
fix_institute_column(example_df).head(20)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,FFCLRP,E,Química - Bacharelado – Habilitações:\rQuímica...,21,13,8,42,9,0,5,5,0,4,4,18
1,FFCLRP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,FFCLRP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,FFCLRP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,FFCLRP,E,Bacharelado em Física Médica,13,9,6,28,7,1,2,3,0,2,2,12
5,FFCLRP,E,Licenciatura e Bacharelado em Química,14,8,6,28,6,0,4,4,0,2,2,12
6,FFCLRP,E,Bacharelado em Matemática Aplicada a\rNegócios,16,10,5,31,6,0,4,4,0,4,4,14
7,FFCLRP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,FFCLRP,B,Psicologia - Bach. e Psicólogo,20,8,0,28,0,0,4,4,0,8,8,12
9,FFCLRP,B,Ciências Biológicas – Bacharelado e\rLicenciatura,20,8,0,28,0,0,4,4,0,8,8,12


### Rename `DataFrame` columns

Fourthly, we rename the columns to compensate for the unusable header which we removed earlier.

In [228]:
def add_column_names(df, names=None):
  if names is not None:
    df.columns = names
  return df

### Encapsulate processing for a single page

Fithly, we encapsulate all the previous four steps into the `process_dataframe` function.

In [261]:
def process_dataframe(df,
                      init_row=None, init_col=None,
                      init_data_col=None,
                      institute_column=None,
                      names=None):
  processed_df = slice_from_df(df, init_row=init_row, init_col=init_col)
  processed_df = remove_cols_without_data(processed_df,
                                          init_data_col=init_data_col)
  processed_df = fix_institute_column(processed_df,
                                      institute_column=institute_column)
  processed_df = add_column_names(processed_df, names=names)
  return processed_df

In [262]:
SLICE = (4, None)
INSTITUTE_COL = 0
INITIAL_DATA_COL = 3
COL_NAMES = ([
  'unidade',
  'area',
  'curso',
  'fuvest_ac',
  'fuvest_ep',
  'fuvest_ppi',
  'fuvest',
  'enem_ac',
  'enem_ep_l1','enem_ep_l3','enem_ep',
  'enem_ppi_l2', 'enem_ppi_l4', 'enem_ppi',
  'enem',
])

processed_df = process_dataframe(raw_df,
                                 init_row=SLICE[0], init_col=SLICE[1],
                                 institute_column=INSTITUTE_COL,
                                 init_data_col=INITIAL_DATA_COL,
                                 names=COL_NAMES)
processed_df

                                                    0  ...                                                  2
0   FFCLRP\rFFCLRP\rFFCLRP\rFFCLRP\rFFCLRP\rFFCLRP...  ...  Química - Bacharelado – Habilitações:\rQuímica...
1                                                 NaN  ...                                                NaN
2                                                 NaN  ...                                                NaN
3                                                 NaN  ...                                                NaN
4                                                 NaN  ...                                                NaN
..                                                ...  ...                                                ...
66                                                 FM  ...                                   Subtotal Unidade
67                                                NaN  ...                                                NaN
68        

,unidade,area,curso,fuvest_ac,fuvest_ep,fuvest_ppi,fuvest,enem_ac,enem_ep_l1,enem_ep_l3,enem_ep,enem_ppi_l2,enem_ppi_l4,enem_ppi,enem
0,FFCLRP,E,Química - Bacharelado – Habilitações:\rQuímica...,21,13,8,42,9,0,5,5,0,4,4,18
1,FFCLRP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,FFCLRP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,FFCLRP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,FFCLRP,E,Bacharelado em Física Médica,13,9,6,28,7,1,2,3,0,2,2,12
5,FFCLRP,E,Licenciatura e Bacharelado em Química,14,8,6,28,6,0,4,4,0,2,2,12
6,FFCLRP,E,Bacharelado em Matemática Aplicada a\rNegócios,16,10,5,31,6,0,4,4,0,4,4,14
7,FFCLRP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,FFCLRP,B,Psicologia - Bach. e Psicólogo,20,8,0,28,0,0,4,4,0,8,8,12
9,FFCLRP,B,Ciências Biológicas – Bacharelado e\rLicenciatura,20,8,0,28,0,0,4,4,0,8,8,12


### Remove remaining rows without data

Sixthly, we remove rows filled with NAs in the data columns which still remain.

In [263]:
def remove_rows_without_data(df, init_data_col=None):
  if init_data_col is not None:
    na_rows = df.iloc[:, init_data_col:].isna().all(axis=1)
    rows_to_remove = na_rows[na_rows==True].index
    return df.drop(rows_to_remove, axis=0)
  return df



In [264]:
remove_rows_without_data(processed_df, init_data_col=3)

,unidade,area,curso,fuvest_ac,fuvest_ep,fuvest_ppi,fuvest,enem_ac,enem_ep_l1,enem_ep_l3,enem_ep,enem_ppi_l2,enem_ppi_l4,enem_ppi,enem
0,FFCLRP,E,Química - Bacharelado – Habilitações:\rQuímica...,21,13,8,42,9,0,5,5,0,4,4,18
4,FFCLRP,E,Bacharelado em Física Médica,13,9,6,28,7,1,2,3,0,2,2,12
5,FFCLRP,E,Licenciatura e Bacharelado em Química,14,8,6,28,6,0,4,4,0,2,2,12
6,FFCLRP,E,Bacharelado em Matemática Aplicada a\rNegócios,16,10,5,31,6,0,4,4,0,4,4,14
8,FFCLRP,B,Psicologia - Bach. e Psicólogo,20,8,0,28,0,0,4,4,0,8,8,12
9,FFCLRP,B,Ciências Biológicas – Bacharelado e\rLicenciatura,20,8,0,28,0,0,4,4,0,8,8,12
11,FFCLRP,H,Pedagogia,25,8,2,35,0,0,7,7,0,8,8,15
12,FFCLRP,H,Bacharelado em Biblioteconomia e Ciência\rda I...,20,6,2,28,0,0,6,6,0,6,6,12
14,FFCLRP,E,Bacharelado em Ciência da Computação,10,4,0,14,0,0,2,2,0,4,4,6
15,FFCLRP,H,Música - Bacharelado e Educação\rArtística - ...,15,9,6,30,0,0,0,0,0,0,0,0


### Encapsulate processing for all pages at once

Finally, encapsulate these steps into the `process_all_pages` function.

In [289]:
def process_all_pages(pages):
  processed_dfs = []
  for i, page in enumerate(pages):
    processed_page = process_dataframe(page,
                                       init_row=SLICE[0], init_col=SLICE[1],
                                       institute_column=INSTITUTE_COL,
                                       init_data_col=INITIAL_DATA_COL,
                                       names=COL_NAMES)
    processed_page = remove_rows_without_data(processed_page,
                                              init_data_col=INITIAL_DATA_COL)
    processed_dfs.append(processed_page)
  final_df = pd.concat(processed_dfs, axis=0, ignore_index=True)

  return final_df.reset_index(drop=True)

In [315]:
final_df = process_all_pages(page_dfs)
final_df

,unidade,area,curso,fuvest_ac,fuvest_ep,fuvest_ppi,fuvest,enem_ac,enem_ep_l1,enem_ep_l3,enem_ep,enem_ppi_l2,enem_ppi_l4,enem_ppi,enem
0,EACH,E,Bacharelado em Sistemas de Informação,28,10,4,42,2,0,8,8,0,8,8,18
1,EACH,E,Bacharelado em Sistemas de Informação,56,21,7,84,4,0,16,16,0,16,16,36
2,EACH,E,Licenciatura em Ciências da Natureza,30,9,3,42,0,0,9,9,0,9,9,18
3,EACH,B,Bacharelado em Educação Física e Saúde,20,14,8,42,10,4,0,4,4,0,4,18
4,EACH,B,Bacharelado em Biotecnologia,30,11,1,42,0,0,7,7,0,11,11,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212,IQSC,E,Química Bacharelado (Hab. Fundamental\re Hab. ...,24,12,6,42,6,3,3,6,3,3,6,18
213,IQSC,NaN,Subtotal Unidade,24,12,6,42,6,3,3,6,3,3,6,18
214,IRI,H,Bacharelado em Relações Internacionais,14,5,2,21,1,3,1,4,3,1,4,9
215,IRI,H,Bacharelado em Relações Internacionais,14,5,2,21,1,3,1,4,3,1,4,9


## Save result to a CSV

In [316]:
final_df.to_csv('../../data/2_intermediate/vagas/2022.csv', index=None)

## Load manually corrected CSV

Some inconsistencies still remain in the CSV resulting from the processing which are easier to fix manually than otherwise.

After these have been corrected, we read the final CSV file.

In [317]:
correct_df = pd.read_csv('../../data/3_formatted/vagas.csv').convert_dtypes()
print(correct_df.dtypes)
correct_df

unidade        string
area           string
curso          string
fuvest_ac       Int64
fuvest_ep       Int64
fuvest_ppi      Int64
fuvest          Int64
enem_ac         Int64
enem_ep_l1      Int64
enem_ep_l3      Int64
enem_ep         Int64
enem_ppi_l2     Int64
enem_ppi_l4     Int64
enem_ppi        Int64
enem            Int64
dtype: object


,unidade,area,curso,fuvest_ac,fuvest_ep,fuvest_ppi,fuvest,enem_ac,enem_ep_l1,enem_ep_l3,enem_ep,enem_ppi_l2,enem_ppi_l4,enem_ppi,enem
0,EACH,E,Bacharelado em Sistemas de Informação,28,10,4,42,2,0,8,8,0,8,8,18
1,EACH,E,Bacharelado em Sistemas de Informação,56,21,7,84,4,0,16,16,0,16,16,36
2,EACH,E,Licenciatura em Ciências da Natureza,30,9,3,42,0,0,9,9,0,9,9,18
3,EACH,B,Bacharelado em Educação Física e Saúde,20,14,8,42,10,4,0,4,4,0,4,18
4,EACH,B,Bacharelado em Biotecnologia,30,11,1,42,0,0,7,7,0,11,11,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212,IQSC,E,Química Bacharelado (Hab. Fundamental e Hab. T...,24,12,6,42,6,3,3,6,3,3,6,18
213,IQSC,<NA>,Subtotal Unidade,24,12,6,42,6,3,3,6,3,3,6,18
214,IRI,H,Bacharelado em Relações Internacionais,14,5,2,21,1,3,1,4,3,1,4,9
215,IRI,H,Bacharelado em Relações Internacionais,14,5,2,21,1,3,1,4,3,1,4,9


## Validate results

Assert that aggregate metrics in each row are correct. All assertions should pass.

In [318]:
assert (
  correct_df.fuvest_ac + correct_df.fuvest_ep + correct_df.fuvest_ppi
  == correct_df.fuvest
).all()

In [319]:
assert(
  correct_df.enem_ep_l1 + correct_df.enem_ep_l3
  == correct_df.enem_ep
).all()

In [320]:
assert(
  correct_df.enem_ppi_l2 + correct_df.enem_ppi_l4
  == correct_df.enem_ppi
).all()

In [321]:
assert(
  correct_df.enem_ac + correct_df.enem_ep + correct_df.enem_ppi
  == correct_df.enem
).all()